<a href="https://colab.research.google.com/github/fcoliveira-utfpr/IDF_equation/blob/main/calibration_IDF_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalando e importando bibliotecas

In [12]:
#Importando bibliotecas
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
from scipy import stats
from scipy.stats import (genextreme, kstest, ksone, lognorm)
from scipy.optimize import minimize
from sklearn.metrics import mean_absolute_error, mean_squared_error

##Importando e abrindo planilha

In [13]:
base_url = "https://raw.githubusercontent.com/fcoliveira-utfpr/IDF_equation/refs/heads/main/chuva_max_7801_xavier.csv"
url = base_url
df = pd.read_csv(url)
df = df.drop(columns=['index', 'Ano'])
df_1 = df.copy()
df_1.head()

,Chuva_Anahy,Chuva_Arapuã,Chuva_Ariranha do Ivaí,Chuva_Boa Esperança,Chuva_Boa Vista da Aparecida,Chuva_Campina da Lagoa,Chuva_Campo Mourão,Chuva_Capanema,Chuva_Capitão Leônidas Marques,Chuva_Céu Azul,...,Chuva_Santa Helena,Chuva_Santa Lúcia,Chuva_Santa Terezinha de Itaipu,Chuva_São José das Palmeiras,Chuva_São Miguel do Iguaçu,Chuva_São Pedro do Iguaçu,Chuva_Serranópolis do Iguaçu,Chuva_Tupãssi,Chuva_Ubiratã,Chuva_Vera Cruz do Oeste
0,76.961569,44.193844,59.932240,66.689038,73.335970,81.575968,51.994375,58.174374,80.147702,53.093042,...,117.227693,71.303437,80.806902,117.337559,71.138637,112.833027,76.247436,68.062371,86.080500,102.615430
1,97.726364,82.235168,85.311434,102.999963,134.037289,95.419165,63.667706,130.631423,135.135955,114.700760,...,114.481027,136.674088,88.552500,128.543957,114.700760,194.903407,129.422890,93.551432,82.509835,160.295416
2,98.165831,74.544503,52.104242,71.413304,148.759419,100.747697,87.014367,99.099697,148.979152,133.487956,...,109.921561,150.956751,89.815966,108.383428,127.005824,124.478891,102.395696,78.884235,111.185028,120.963159
3,95.309298,70.644237,97.726364,84.542367,60.975973,103.549296,69.490638,85.366367,62.349306,100.582897,...,69.106105,75.203703,74.819170,71.687971,69.490638,65.315705,60.756240,74.764236,79.598369,64.326906
4,77.565836,54.631175,51.005576,81.960501,46.610910,95.364231,66.249572,78.389836,43.287444,54.493841,...,73.830370,45.292510,61.305573,76.577036,57.075707,52.653575,61.305573,108.987695,78.225036,53.010642


##Desagregação da chuva máxima diária

In [17]:
# Criando um dicionário com os coeficientes de desagregação
fatores = {
    "5 min": 0.34, "10 min": 0.54, "15 min": 0.70, "20 min": 0.81, "30 min": 0.74,
    "60 min": 0.42, "360 min": 0.72, "480 min": 0.78, "600 min": 0.82, "720 min": 0.85,
    "1440 min": 1.14
}

# Criando um DataFrame para armazenar os resultados
dfs_desagregados = []

# Aplicando os fatores de desagregação para cada coluna em df_1
for coluna in df_1.columns:
    df_des = df_1[[coluna]].rename(columns={coluna: "Chuva (mm)"})

    # Aplicando os fatores de desagregação
    df_des['1440 min'] = df_des['Chuva (mm)'] * fatores['1440 min']
    df_des['720 min'] = df_des['1440 min'] * fatores['720 min']
    df_des['600 min'] = df_des['1440 min'] * fatores['600 min']
    df_des['480 min'] = df_des['1440 min'] * fatores['480 min']
    df_des['360 min'] = df_des['1440 min'] * fatores['360 min']
    df_des['60 min'] = df_des['1440 min'] * fatores['60 min']
    df_des['30 min'] = df_des['60 min'] * fatores['30 min']
    df_des['20 min'] = df_des['30 min'] * fatores['20 min']
    df_des['15 min'] = df_des['30 min'] * fatores['15 min']
    df_des['10 min'] = df_des['30 min'] * fatores['10 min']
    df_des['5 min'] = df_des['30 min'] * fatores['5 min']

    # Removendo a coluna original de chuva diária
    df_des = df_des.drop(columns=['Chuva (mm)'])

    # Adicionando um identificador da coluna original
    df_des['Municipio'] = coluna

    # Armazenando no DataFrame de resultados
    dfs_desagregados.append(df_des)

# Concatenando todos os DataFrames resultantes
resultado = pd.concat(dfs_desagregados, ignore_index=True)
# Removendo a palavra "Chuva_" da coluna "Municipio"
resultado['Municipio'] = resultado['Municipio'].str.replace('Chuva_', '', regex=False)
resultado

,1440 min,720 min,600 min,480 min,360 min,60 min,30 min,20 min,15 min,10 min,5 min,Municipio
0,87.736189,74.575761,71.943675,68.434227,63.170056,36.849199,27.268408,22.087410,19.087885,14.724940,9.271259,Anahy
1,111.408055,94.696847,91.354605,86.898283,80.213800,46.791383,34.625624,28.046755,24.237936,18.697837,11.772712,Anahy
2,111.909047,95.122690,91.765419,87.289057,80.574514,47.001800,34.781332,28.172879,24.346932,18.781919,11.825653,Anahy
3,108.652600,92.354710,89.095132,84.749028,78.229872,45.634092,33.769228,27.353075,23.638460,18.235383,11.481538,Anahy
4,88.425053,75.161295,72.508543,68.971541,63.666038,37.138522,27.482506,22.260830,19.237754,14.840553,9.344052,Anahy
...,...,...,...,...,...,...,...,...,...,...,...,...
1483,180.231815,153.197042,147.790088,140.580815,129.766906,75.697362,56.016048,45.372999,39.211234,30.248666,19.045456,Vera Cruz do Oeste
1484,93.560220,79.526187,76.719380,72.976971,67.363358,39.295292,29.078516,23.553598,20.354961,15.702399,9.886696,Vera Cruz do Oeste
1485,65.128930,55.359591,53.405723,50.800566,46.892830,27.354151,20.242072,16.396078,14.169450,10.930719,6.882304,Vera Cruz do Oeste
1486,101.075098,85.913833,82.881580,78.838576,72.774070,42.451541,31.414140,25.445454,21.989898,16.963636,10.680808,Vera Cruz do Oeste


##Calculando as intensidade máximas

In [ ]:
#Critério para calcular a intensidade de aplicação
p_i = {
    "5 min": (5/60), "10 min": (10/60), "15 min": (15/60), "20 min": (20/60), "30 min": (30/60),
    "60 min": 1, "360 min": 6, "480 min": 8, "600 min": 10, "720 min": 12,
    "1440 min": 24
}

In [ ]:
# Criar um novo DataFrame i_max
i_max = resultado.copy()
municipios = i_max['Municipio']
i_max = i_max.apply(lambda x: pd.to_numeric(x, errors = 'coerce'), axis=1)
i_max.drop(columns=['Municipio'], inplace=True)

# Dividir cada coluna pelo valor correspondente no dicionário p_i
for coluna in i_max.columns:
    i_max[coluna] = i_max[coluna] / p_i[coluna]

# Renomear as colunas para refletir a operação realizada
i_max.columns = [f"i_max_{col}" for col in i_max.columns]
i_max['Municipio'] = municipios
i_max

,i_max_1440 min,i_max_720 min,i_max_600 min,i_max_480 min,i_max_360 min,i_max_60 min,i_max_30 min,i_max_20 min,i_max_15 min,i_max_10 min,i_max_5 min,Municipio
0,3.655675,5.451444,6.310849,7.503753,9.235388,32.323859,113.903123,138.392294,159.464372,184.523058,232.362370,Anahy
1,4.642002,6.922284,8.013562,9.528321,11.727164,41.045073,144.635019,175.731548,202.489027,234.308731,295.055439,Anahy
2,4.662877,6.953413,8.049598,9.571169,11.779900,41.229649,145.285430,176.521797,203.399601,235.362396,296.382276,Anahy
3,4.527192,6.751075,7.815362,9.292657,11.437116,40.029905,141.057761,171.385180,197.480866,228.513573,287.757833,Anahy
4,3.684377,5.494247,6.360399,7.562669,9.307900,32.577651,114.797437,139.478886,160.716412,185.971848,234.186771,Anahy
...,...,...,...,...,...,...,...,...,...,...,...,...
1483,7.509659,11.198614,12.964043,15.414563,18.971770,66.401195,233.985163,284.291973,327.579228,379.055964,477.329732,Vera Cruz do Oeste
1484,3.898342,5.813318,6.729770,8.001861,9.848444,34.469555,121.464145,147.578936,170.049803,196.771914,247.786855,Vera Cruz do Oeste
1485,2.713705,4.046754,4.684713,5.570237,6.855677,23.994869,84.553348,102.732318,118.374688,136.976424,172.488831,Vera Cruz do Oeste
1486,4.211462,6.280251,7.270314,8.644581,10.639484,37.238194,131.220302,159.432667,183.708423,212.576890,267.689417,Vera Cruz do Oeste


##Avaliar a aderências das FDP aos dados

In [ ]:
# Função para ajustar as distribuições e retornar os resultados
def ajustar_distribuicoes(chuva_anual, nome_serie):
    # Crie um intervalo de valores para a precipitação
    x = np.linspace(0, chuva_anual[~pd.isnull(chuva_anual)].astype(float).max(), 1000)

    # Função para ajustar a distribuição Log-Pearson III
    def log_pearson3_fit(data):
        log_data = np.log(data)
        shape, loc, scale = stats.pearson3.fit(log_data)
        return shape, loc, scale

    ## Gumbel (Extreme Value Type I)
    params_gumbel = stats.gumbel_r.fit(chuva_anual)
    D_gumbel, p_value_gumbel = kstest(chuva_anual, 'gumbel_r', args=params_gumbel)

    ## Log-Normal
    params_ln = stats.lognorm.fit(chuva_anual)
    D_ln, p_value_ln = kstest(chuva_anual, 'lognorm', args=params_ln)

    ## GEV (Generalized Extreme Value)
    params_gev = genextreme.fit(chuva_anual)
    D_gev, p_value_gev = kstest(chuva_anual, 'genextreme', args=params_gev)

    ## Log-Pearson III
    shape_lp3, loc_lp3, scale_lp3 = log_pearson3_fit(chuva_anual)
    D_lp3, p_value_lp3 = kstest(np.log(chuva_anual), 'pearson3', args=(shape_lp3, loc_lp3, scale_lp3))

    # Cálculo do D crítico
    n = len(chuva_anual)
    alpha = 0.05  # Nível de significância
    D_critical = ksone.ppf(1 - alpha / 2, n)

    # Crie um dicionário com os dados
    data = {
        "Série": [nome_serie] * 4,  # Repete o nome da série para cada distribuição
        "Distribuição": ["Gumbel", "Log-Normal", "GEV", "Log-Pearson III"],
        "P-valor": [p_value_gumbel, p_value_ln, p_value_gev, p_value_lp3],
        "Dsup": [D_gumbel, D_ln, D_gev, D_lp3],
        "Dcrítico": [D_critical] * 4,  # Valor de D calculado (Dsup) tem que ser menor que D crítico.
        "Coeficientes": [
            f"loc={params_gumbel[0]:.1f}, scale={params_gumbel[1]:.1f}",
            f"shape={params_ln[0]:.1f}, loc={params_ln[1]:.1f}, scale={params_ln[2]:.1f}",
            f"c={params_gev[0]:.1f}, loc={params_gev[1]:.1f}, scale={params_gev[2]:.1f}",
            f"shape={shape_lp3:.1f}, loc={loc_lp3:.1f}, scale={scale_lp3:.1f}",
        ]
    }

    # Crie o DataFrame
    result = pd.DataFrame(data)

    # Determine o melhor ajuste com base no nível de significância
    result["Melhor Ajuste"] = "Não significativo"  # Começa com "Não significativo" e é atualizado se um ajuste for adequado
    result.loc[result["P-valor"] > alpha, "Melhor Ajuste"] = result.loc[result["P-valor"] > alpha, "Distribuição"]

    # Ordene os resultados pelo Dsup
    result = result.sort_values(by='Dsup').reset_index(drop=True)
    result = result[['Série', 'Melhor Ajuste', 'P-valor', 'Dsup', 'Dcrítico', 'Coeficientes']]

    return result

# Função para ajustar a distribuição Log-Pearson III
def log_pearson3_fit(data):
    log_data = np.log(data)
    shape, loc, scale = stats.pearson3.fit(log_data)
    return shape, loc, scale

In [ ]:
#Lista de cidades
cidades = municipios.unique()

# DataFrame final que irá armazenar os resultados de todas as cidades
resultados_finais_todas_cidades = pd.DataFrame()

# Iterar sobre cada cidade na lista
for cidade in cidades:
    # Filtrar o DataFrame i_max para a cidade atual
    df_max = i_max[i_max['Municipio'] == cidade].copy()

    # Remover a coluna 'Municipio' pois não será mais necessária
    df_max.drop(columns=['Municipio'], inplace=True)

    # DataFrame temporário para armazenar os resultados da cidade atual
    resultados_finais = pd.DataFrame()

    # Aplicar a função para cada coluna do df_max
    for coluna in df_max.columns:
        chuva_anual = df_max[coluna].values
        resultado = ajustar_distribuicoes(chuva_anual, coluna)
        resultados_finais = pd.concat([resultados_finais, resultado], ignore_index=True)

    # Adicionar a coluna 'Municipio' ao DataFrame de resultados da cidade atual
    resultados_finais['Municipio'] = cidade

    # Concatenar os resultados da cidade atual ao DataFrame final de todas as cidades
    resultados_finais_todas_cidades = pd.concat([resultados_finais_todas_cidades, resultados_finais], ignore_index=True)

# Exibir o DataFrame final com os resultados de todas as cidades
resultados_finais_todas_cidades

,Série,Melhor Ajuste,P-valor,Dsup,Dcrítico,Coeficientes,Municipio
0,i_max_1440 min,Log-Normal,0.771094,0.114273,0.237884,"shape=0.2, loc=0.2, scale=3.9",Anahy
1,i_max_1440 min,Log-Pearson III,0.762821,0.115197,0.237884,"shape=0.1, loc=1.4, scale=0.2",Anahy
2,i_max_1440 min,GEV,0.752859,0.116300,0.237884,"c=0.1, loc=3.7, scale=0.9",Anahy
3,i_max_1440 min,Gumbel,0.650195,0.127306,0.237884,"loc=3.7, scale=0.9",Anahy
4,i_max_720 min,Log-Normal,0.771094,0.114273,0.237884,"shape=0.2, loc=0.2, scale=5.8",Anahy
...,...,...,...,...,...,...,...
2107,i_max_10 min,Log-Normal,0.874314,0.101608,0.237884,"shape=0.4, loc=59.3, scale=169.1",Vera Cruz do Oeste
2108,i_max_5 min,Gumbel,0.957527,0.086848,0.237884,"loc=262.3, scale=76.7",Vera Cruz do Oeste
2109,i_max_5 min,Log-Pearson III,0.893016,0.098907,0.237884,"shape=0.3, loc=5.7, scale=0.3",Vera Cruz do Oeste
2110,i_max_5 min,Log-Normal,0.874314,0.101608,0.237884,"shape=0.4, loc=74.7, scale=213.0",Vera Cruz do Oeste


##Determinando as intensidades máximas para cada duração



In [ ]:
# Função para calcular precipitação para cada tipo de distribuição
def calcular_precipitacao(chuva_anual, probabilidade, tipo_distribuicao):
    if tipo_distribuicao == "Log-Normal":
        shape_ln, loc_ln, scale_ln = stats.lognorm.fit(chuva_anual)
        return max(0, stats.lognorm.ppf(probabilidade, shape_ln, loc_ln, scale_ln))

    elif tipo_distribuicao == "Gumbel":
        loc_gumbel, scale_gumbel = stats.gumbel_r.fit(chuva_anual)
        return max(0, stats.gumbel_r.ppf(probabilidade, loc_gumbel, scale_gumbel))

    elif tipo_distribuicao == "GEV":
        c_gev, loc_gev, scale_gev = stats.genextreme.fit(chuva_anual)
        return max(0, stats.genextreme.ppf(probabilidade, c_gev, loc_gev, scale_gev))

    elif tipo_distribuicao == "Log-Pearson III":
        # A Log-Pearson III é baseada em distribuição normal após logaritmo
        shape_lp, loc_lp, scale_lp = stats.lognorm.fit(np.log(chuva_anual))
        return max(0, stats.lognorm.ppf(probabilidade, shape_lp, loc_lp, scale_lp))

    else:
        raise ValueError("Distribuição não reconhecida. Escolha entre 'Gumbel', 'Log-Normal', 'GEV', 'Log-Pearson III'.")


In [ ]:
#Lista de cidades
cidades = municipios.unique()

# Lista para armazenar os DataFrames de cada cidade
dfs_resultados = []

for cidade in cidades:
    df_2 = resultados_finais_todas_cidades[resultados_finais_todas_cidades['Municipio'] == cidade].copy()
    i_max_x = i_max[i_max['Municipio'] == cidade].copy()
    i_max_x = i_max_x.drop(columns=['Municipio'], inplace=False)

    # Solicita o tipo de distribuição
    distribuicao = df_2['Melhor Ajuste'].iloc[0]

    # Períodos de retorno
    periodos_retorno = list(range(2, 101, 2))

    # Lista de níveis de probabilidade
    nivel_probabilidade = [(1 / t) for t in periodos_retorno]

    # DataFrame para armazenar os resultados finais
    df_resultados = pd.DataFrame({"T (anos)": periodos_retorno})

    # Iterar sobre cada coluna de i_max
    for coluna in i_max_x.columns:
        chuva_anual = i_max_x[coluna]
        precipitacao_provavel = []

        # Calcular a precipitação provável para cada nível de probabilidade
        for probabilidade in nivel_probabilidade:
            precipitacao_provavel_anual = calcular_precipitacao(chuva_anual, probabilidade, distribuicao)
            precipitacao_provavel.append(precipitacao_provavel_anual)

        # Adicionar os resultados ao DataFrame final
        nome_coluna_resultado = coluna.split()[0]
        df_resultados[nome_coluna_resultado] = precipitacao_provavel

    # Organizando o df
    df_resultados.drop(columns=['T (anos)'], inplace=True)
    df_i_max = df_resultados.T
    df_i_max = df_i_max.reset_index()
    colunas_tempo_invertida = periodos_retorno[::-1]
    df_i_max.columns = ['t (min)'] + [f'{valor} anos' for valor in colunas_tempo_invertida]
    df_i_max['Municipio'] = cidade

    # Armazena o resultado da cidade
    dfs_resultados.append(df_i_max)

# Concatenando todos os resultados
df_fin = pd.concat(dfs_resultados, ignore_index=True)
df_fin

,t (min),100 anos,98 anos,96 anos,94 anos,92 anos,90 anos,88 anos,86 anos,84 anos,...,18 anos,16 anos,14 anos,12 anos,10 anos,8 anos,6 anos,4 anos,2 anos,Municipio
0,i_max_1440,4.036008,3.436081,3.205365,3.069609,2.975984,2.905667,2.849953,2.804156,2.765489,...,2.366421,2.361468,2.356654,2.351973,2.347417,2.342981,2.338659,2.334446,2.330336,Anahy
1,i_max_720,6.018608,5.123980,4.779930,4.577487,4.437871,4.333013,4.249929,4.181636,4.123974,...,3.528873,3.521487,3.514309,3.507328,3.500534,3.493919,3.487474,3.481191,3.475062,Anahy
2,i_max_600,6.967424,5.931761,5.533472,5.299114,5.137489,5.016099,4.919918,4.840858,4.774107,...,4.085190,4.076639,4.068329,4.060247,4.052383,4.044725,4.037264,4.029990,4.022896,Anahy
3,i_max_480,8.284437,7.053008,6.579433,6.300776,6.108599,5.964264,5.849903,5.755899,5.676529,...,4.857390,4.847223,4.837342,4.827733,4.818382,4.809277,4.800405,4.791757,4.783321,Anahy
4,i_max_360,10.196230,8.680626,8.097764,7.754801,7.518276,7.340633,7.199880,7.084183,6.986497,...,5.978326,5.965813,5.953652,5.941825,5.930316,5.919110,5.908191,5.897547,5.887165,Anahy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,i_max_30,142.341202,116.284794,106.640187,101.042772,97.210959,94.346391,92.083886,90.228385,88.664442,...,72.605418,72.406312,72.212796,72.024587,71.841422,71.663056,71.489261,71.319826,71.154551,Vera Cruz do Oeste
524,i_max_20,172.944561,141.286024,129.567827,122.766968,118.111316,114.630865,111.881922,109.627488,107.727297,...,88.215583,87.973669,87.738547,87.509873,87.287328,87.070613,86.859453,86.653588,86.452779,Vera Cruz do Oeste
525,i_max_15,199.277683,162.798711,149.296262,141.459880,136.095343,132.084948,128.917441,126.319739,124.130219,...,101.647586,101.368836,101.097914,100.834422,100.577991,100.328278,100.084966,99.847756,99.616371,Vera Cruz do Oeste
526,i_max_10,230.592748,188.381366,172.757103,163.689290,157.481754,152.841154,149.175896,146.169984,143.636396,...,117.620778,117.298225,116.984729,116.679831,116.383104,116.094151,115.812604,115.538118,115.270372,Vera Cruz do Oeste


##Calibrando coeficientes maximizando R²

In [ ]:
#Lista de cidades
cidades = municipios.unique()

# Lista para armazenar os resultados de cada cidade
dfs_resultados = []

for cidade in cidades:
    df_i_max = df_fin[df_fin['Municipio'] == cidade].copy()
    df_i_max = df_i_max.drop(columns=['Municipio'])
    df_i_max['t (min)'] = df_i_max['t (min)'].str.replace('i_max_', '')
    df_i_max['t (min)'] = pd.to_numeric(df_i_max['t (min)'], errors='coerce')

    # Extração de tempo (t) e intensidades observadas
    t = df_i_max.iloc[:, 0].values
    periodos_retorno = list(range(2, 101, 2))
    intensidades_obs = {T: df_i_max[f"{T} anos"].values for T in periodos_retorno}

    #Transformando os dados
    intensidades_obs_log = {T: np.log(intensidades_obs[T]) for T in periodos_retorno}
    intensidades_obs = intensidades_obs_log

    # Definir a equação IDF
    def equacao_IDF(t, k, a, b, c, T):
        return (k * T**a) / ((t + b)**c)

    # Função de erro (negativo do R²) para maximizar
    def erro(params):
        k, a, b, c = params
        intensidades_est_combined = []
        intensidades_obs_combined = []
        for T in periodos_retorno:
            I_pred = equacao_IDF(t, k, a, b, c, T)
            intensidades_est_combined.extend(I_pred)
            intensidades_obs_combined.extend(intensidades_obs[T])

        # Calcular R²
        ss_total = np.sum((np.array(intensidades_obs_combined) - np.mean(intensidades_obs_combined))**2)
        ss_residual = np.sum((np.array(intensidades_obs_combined) - np.array(intensidades_est_combined))**2)
        r_square_total = 1 - (ss_residual / ss_total)

        return -r_square_total  # Maximizar R²

    # Parâmetros iniciais e limites
    parametros_iniciais = [2000, 0.1, 10, 0.5]
    bounds = [(100, 5000), (0.001, 1), (1, 100), (0.001, 1)]  # k, a, b, c

    # Otimização
    resultado = minimize(erro, parametros_iniciais, method='TNC', bounds=bounds)
    coef_otimizados = resultado.x

    # Criar DataFrame com coeficientes
    coeficientes_df = pd.DataFrame([coef_otimizados], columns=['k', 'a', 'b', 'c'])

    # Cálculo dos erros (MAE, RMSE, R²)
    k_ot, a_ot, b_ot, c_ot = coef_otimizados
    intensidades_est = {T: equacao_IDF(t, k_ot, a_ot, b_ot, c_ot, T) for T in periodos_retorno}
    intensidades_obs_combined = np.concatenate([intensidades_obs[T] for T in periodos_retorno])
    intensidades_est_combined = np.concatenate([intensidades_est[T] for T in periodos_retorno])

    mae_total = mean_absolute_error(intensidades_obs_combined, intensidades_est_combined)
    rmse_total = np.sqrt(mean_squared_error(intensidades_obs_combined, intensidades_est_combined))
    ss_total = np.sum((intensidades_obs_combined - np.mean(intensidades_obs_combined))**2)
    ss_residual = np.sum((intensidades_obs_combined - intensidades_est_combined)**2)
    r_square_total = 1 - (ss_residual / ss_total)

    # Criar DataFrame de resultados
    resultados_aggregados = pd.DataFrame([{
        'MAE': mae_total,
        'RMSE': rmse_total,
        'R²': r_square_total
    }])

    # Concatenar coeficientes e métricas
    df_res = pd.concat([coeficientes_df, resultados_aggregados], axis=1)
    df_res['Municipio'] = cidade

    # Armazena os resultados da cidade
    dfs_resultados.append(df_res)

# Concatenando todos os resultados
df_final = pd.concat(dfs_resultados, ignore_index=True)
df_final


<ipython-input-11-89635459fe93>:19: RuntimeWarning: divide by zero encountered in log
  intensidades_obs_log = {T: np.log(intensidades_obs[T]) for T in periodos_retorno}
<ipython-input-11-89635459fe93>:37: RuntimeWarning: invalid value encountered in subtract
  ss_total = np.sum((np.array(intensidades_obs_combined) - np.mean(intensidades_obs_combined))**2)


ValueError: Input contains infinity or a value too large for dtype('float64').

#teste

In [ ]:
!pip install scikit-optimize    -q

In [ ]:
#Lista de cidades
cidades = municipios.unique()

# Lista para armazenar os resultados de cada cidade
dfs_resultados = []

for cidade in cidades:
    df_i_max = df_fin[df_fin['Municipio'] == cidade].copy()
    df_i_max = df_i_max.drop(columns=['Municipio'])
    df_i_max['t (min)'] = df_i_max['t (min)'].str.replace('i_max_', '')
    df_i_max['t (min)'] = pd.to_numeric(df_i_max['t (min)'], errors='coerce')

    # Extração de tempo (t) e intensidades observadas
    t = df_i_max.iloc[:, 0].values
    periodos_retorno = list(range(2, 101, 2))
    intensidades_obs = {T: df_i_max[f"{T} anos"].values for T in periodos_retorno}

    #Transformando os dados
    intensidades_obs_log = {T: np.log(intensidades_obs[T]) for T in periodos_retorno}
    intensidades_obs = intensidades_obs_log

    # Definir a equação IDF
    def equacao_IDF(t, k, a, b, c, T):
        return (k * T**a) / ((t + b)**c)

    # Import the gp_minimize function
    from skopt import gp_minimize

    # Função de erro
    def erro(params):
        k, a, b, c = params
        intensidades_est_combined = []
        intensidades_obs_combined = []
        for T in periodos_retorno:
            I_pred = equacao_IDF(t, k, a, b, c, T) # Corrected indentation
            intensidades_est_combined.extend(I_pred) # Corrected indentation
            intensidades_obs_combined.extend(intensidades_obs[T]) # Corrected indentation
        ss_total = np.sum((np.array(intensidades_obs_combined) - np.mean(intensidades_obs_combined))**2)
        ss_residual = np.sum((np.array(intensidades_obs_combined) - np.array(intensidades_est_combined))**2)
        r_square_total = 1 - (ss_residual / ss_total)
        return -r_square_total

    # Espaço de busca para os parâmetros
    space = [(100, 5000), (0.001, 1), (1, 100), (0.001, 1)]

    # Otimização Bayesiana
    resultado = gp_minimize(erro, space, n_calls=50, random_state=42)
    coef_otimizados = resultado.x

    # Criar DataFrame com coeficientes
    coeficientes_df = pd.DataFrame([coef_otimizados], columns=['k', 'a', 'b', 'c'])

    # Cálculo dos erros (MAE, RMSE, R²)
    k_ot, a_ot, b_ot, c_ot = coef_otimizados
    intensidades_est = {T: equacao_IDF(t, k_ot, a_ot, b_ot, c_ot, T) for T in periodos_retorno}
    intensidades_obs_combined = np.concatenate([intensidades_obs[T] for T in periodos_retorno])
    intensidades_est_combined = np.concatenate([intensidades_est[T] for T in periodos_retorno])

    mae_total = mean_absolute_error(intensidades_obs_combined, intensidades_est_combined)
    rmse_total = np.sqrt(mean_squared_error(intensidades_obs_combined, intensidades_est_combined))
    ss_total = np.sum((intensidades_obs_combined - np.mean(intensidades_obs_combined))**2)
    ss_residual = np.sum((intensidades_obs_combined - intensidades_est_combined)**2)
    r_square_total = 1 - (ss_residual / ss_total)

    # Criar DataFrame de resultados
    resultados_aggregados = pd.DataFrame([{
        'MAE': mae_total,
        'RMSE': rmse_total,
        'R²': r_square_total
    }])

    # Concatenar coeficientes e métricas
    df_res = pd.concat([coeficientes_df, resultados_aggregados], axis=1)
    df_res['Municipio'] = cidade

    # Armazena os resultados da cidade
    dfs_resultados.append(df_res)

# Concatenando todos os resultados
df_final = pd.concat(dfs_resultados, ignore_index=True)
df_final

In [ ]:
# Lista de cidades
cidades = municipios.unique()

# Lista para armazenar os resultados de cada cidade
dfs_resultados = []

for cidade in cidades:
    df_i_max_cidade = df_fin[df_fin['Municipio'] == cidade].copy()
    df_i_max_cidade['t (min)'] = df_i_max_cidade['t (min)'].str.replace('i_max_', '')
    df_i_max_cidade['t (min)'] = pd.to_numeric(df_i_max_cidade['t (min)'], errors='coerce')

    # Extrair tempo (t) e períodos de retorno (T)
    t = df_i_max_cidade.iloc[:, 0].values
    periodos_retorno = list(range(2, 101, 2))

    # Extrair intensidades observadas
    intensidades_obs = {T: df_i_max_cidade[f"{T} anos"].values for T in periodos_retorno}

    # Função de erro para Monte Carlo
    def erro_montecarlo(params, t, periodos_retorno, intensidades_obs):
        k, a, b, c = params
        intensidades_est_combined = []
        intensidades_obs_combined = []

        for T in periodos_retorno:
            I_pred = equacao_IDF(t, k, a, b, c, T)
            intensidades_est_combined.extend(I_pred)
            intensidades_obs_combined.extend(intensidades_obs[T])

        ss_total = np.sum((np.array(intensidades_obs_combined) - np.mean(intensidades_obs_combined))**2)
        ss_residual = np.sum((np.array(intensidades_obs_combined) - np.array(intensidades_est_combined))**2)
        return - (1 - (ss_residual / ss_total))  # Retorna o negativo do R²

    # Definir os limites para os parâmetros k, a, b, c
    bounds = {
        'k': (500, 5000),
        'a': (0, 1),
        'b': (1, 100),
        'c': (0, 1)
    }

    num_iter = 10000  # Número de iterações do Monte Carlo
    melhor_erro = np.inf
    melhores_parametros = None

    for i in range(num_iter):
        k = np.random.uniform(*bounds['k'])
        a = np.random.uniform(*bounds['a'])
        b = np.random.uniform(*bounds['b'])
        c = np.random.uniform(*bounds['c'])

        erro_atual = erro_montecarlo([k, a, b, c], t, periodos_retorno, intensidades_obs)

        if erro_atual < melhor_erro:
            melhor_erro = erro_atual
            melhores_parametros = [k, a, b, c]

    k_ot, a_ot, b_ot, c_ot = melhores_parametros
    intensidades_est = {T: equacao_IDF(t, k_ot, a_ot, b_ot, c_ot, T) for T in periodos_retorno}

    intensidades_obs_combined = np.concatenate([intensidades_obs[T] for T in periodos_retorno])
    intensidades_est_combined = np.concatenate([intensidades_est[T] for T in periodos_retorno])

    mae_total = mean_absolute_error(intensidades_obs_combined, intensidades_est_combined)
    rmse_total = np.sqrt(mean_squared_error(intensidades_obs_combined, intensidades_est_combined))

    ss_total = np.sum((intensidades_obs_combined - np.mean(intensidades_obs_combined))**2)
    ss_residual = np.sum((intensidades_obs_combined - intensidades_est_combined)**2)
    r_square_total = 1 - (ss_residual / ss_total)

    resultados_aggregados = {
        'MAE': mae_total,
        'RMSE': rmse_total,
        'R²': r_square_total
    }

    coeficientes_df = pd.DataFrame([melhores_parametros], columns=['k', 'a', 'b', 'c'])
    res = pd.DataFrame([resultados_aggregados])
    df_res = pd.concat([coeficientes_df, res], axis=1)
    df_res['Municipio'] = cidade

    dfs_resultados.append(df_res)

df_final = pd.concat(dfs_resultados, ignore_index=True)
df_final
